# (Close to) Online Event Viewer

This notebook can be used to view GRAW data without merging the files ahead of time.

Run cells by pressing `shift-return`. Reset the notebook (and clear variables) by pressing `ESC 0 0` (those are zeros)

## Imports and Definitions

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
from scipy.stats import threshold
import warnings
import copy
import IPython.display
import sys
import os

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pytpc
from pytpc.constants import *

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
sns.set_color_codes()

In [ ]:
def subtract_and_delete_fpn(traces):
    """Subtract the normalized, average fixed-pattern noise from the data.
    
    The FPN channels for each AGET are averaged, renormalized to zero, and subtracted
    from the data signals in that AGET. They are then deleted from the data.
    
    Parameters
    ----------
    traces : np.ndarray
        The structured NumPy array from the event, e.g. `evt.traces`
    
    Returns
    -------
    ndarray
        The same structured array as before, but with the FPN subtracted and deleted.
    
    """
    
    traces = copy.deepcopy(traces)
    
    fpn_channels = [11, 22, 45, 56]
    
    for cobo, asad, aget in {tuple(a) for a in traces[['cobo', 'asad', 'aget']]}:
        fpn_idx = np.where(np.all((traces['cobo'] == cobo, traces['asad'] == asad, 
                                   traces['aget'] == aget, np.in1d(traces['channel'], fpn_channels)), axis=0))[0]
        data_idx = np.where(np.all((traces['cobo'] == cobo, traces['asad'] == asad, 
                                    traces['aget'] == aget, ~np.in1d(traces['channel'], fpn_channels)), axis=0))[0]

        if len(fpn_idx) != 4:
            warnings.warn('Number of FPN channels was incorrect: %d (should be 4)' % len(idx))

        mean_fpn = traces['data'][fpn_idx].mean(axis=0)
        mean_fpn -= mean_fpn.mean()

        traces['data'][data_idx] -= mean_fpn
    
    return np.delete(traces, np.where(np.in1d(traces['channel'], fpn_channels)))

## Pad mapping

Import the pad mapping here. This should be a CSV file with columns

    CoBo , AsAd , AGET , Channel , Pad

In [ ]:
padmap = pd.read_csv(os.path.expanduser('~/Dropbox/routing/Lookup20141208.csv'), header=None,
                     names=('cobo', 'asad', 'aget', 'channel', 'pad'), index_col=-1)

Make a dictionary to facilitate reverse lookup of pad number from hardware address.

In [ ]:
pad_lookup = {(cobo, asad, aget, ch) : pad for pad, cobo, asad, aget, ch in padmap.itertuples()}

## GRAW Files

Here is where we read the GRAW files. The `glob()` function will match any set of files using a syntax like in `bash`.

In [ ]:
# The paths of the GRAW files
gfile_paths = glob.glob('/Users/josh/Documents/Data/Old-Alphas-Dec14/run_0225/*.graw')
print('Found {} GRAW files'.format(len(gfile_paths)))

This is where the files are actually read. This could take a bit since each file needs to be indexed.

In [ ]:
gfiles = []

for i, p in enumerate(gfile_paths):
    IPython.display.clear_output()
    print('Opening file {} / {}'.format(i+1, len(gfile_paths)))
    sys.stdout.flush()
    gfiles.append(pytpc.grawdata.GRAWFile(p, max_len=100))
    
print('Done')

## Read events

The next cells read the events and merge them. See the comments line-by-line to enable or disable specific steps.

Individual lines can be commented out by clicking somewhere on them and pressing `cmd-/`

In [ ]:
event_number = 10  # This event will be read

In [ ]:
# Read the event and merge the frames
evt = pytpc.grawdata.merge_frames(gfiles, event_number)

# Apply the pad map to the data
evt.traces['pad'] = [pad_lookup.get(tuple(a), 20000) for a in evt.traces[['cobo', 'asad', 'aget', 'channel']]]

# Subtract and remove the fixed-pattern noise
evt.traces = subtract_and_delete_fpn(evt.traces)

# Subtract an ad hoc pedestal (the median value of the trace)
evt.traces['data'] = (evt.traces['data'].T - np.median(evt.traces['data'], axis=1)).T

# Apply a threshold to the trace
evt.traces['data'] = threshold(evt.traces['data'], threshmin=40.)

## Plots

Note that under each plot, there is a line to save the plot to a file. If you un-comment this line, it will be saved. Change the path to whatever you like.

### Mesh

In [ ]:
plt.plot(evt.traces['data'].sum(0))
sns.despine()
plt.xlim(0, 512)
plt.xlabel('Time bucket')
plt.ylabel('Total activation')
# plt.savefig(os.path.expanduser('~/Desktop/mesh.png'), transparent=False, dpi=100)

### Pad plane

In [ ]:
pplane = pytpc.pad_plot(evt.hits())
# pplane.savefig(os.path.expanduser('~/Desktop/pad_plane.png'), transparent=True, dpi=100)

### 3-D View

In [ ]:
chplot = pytpc.chamber_plot(evt.xyzs())
chplot.axes[0].view_init(azim=0, elev=20)  # Change these angles to rotate the view
# chplot.savefig(os.path.expanduser('~/Desktop/chamber.png'), transparent=False, dpi=100)

### Individual Traces

In [ ]:
plt.plot(evt.traces['data'].T, lw=1)
# plt.plot(evt.traces['data'].sum(0), 'k--')          # uncomment this line to plot the sum too
sns.despine()
plt.xlim(0, 512)
plt.xlabel('Time bucket')
plt.ylabel('Sample value')
# bokeh.plotting.show(bokeh.mpl.to_bokeh())
# plt.savefig(os.path.expanduser('~/Desktop/traces.png'), transparent=False, dpi=100)